In [16]:
## Installations for the model
!pip install -q transformers accelerate datasets[audio]
#!pip install -q flash-attn --no-build-isolation
!pip install --q optimum


In [1]:
## Installations for handling audio and video files
%%capture
!pip install yt-dlp
!pip install -q moviepy librosa
!pip install -q ffmpeg moviepy
!pip install -q pydub

In [ ]:
## url = https://www.youtube.com/watch?v=2rtSSM-C91I

In [2]:
import os
from yt_dlp import YoutubeDL

def download_video(video_url, output_dir="downloads"):
    """
    Downloads a video from the given URL in the best quality.

    Args:
        video_url (str): URL of the video to download.
        output_dir (str): Directory where the video will be saved.
    """
    # Ensure output directory exists
    os.makedirs(output_dir, exist_ok=True)

    # Configure yt-dlp options
    ydl_opts = {
        'format': 'bestvideo+bestaudio/best',  # Downloads best video + audio or best combined
        'outtmpl': os.path.join(output_dir, '%(title)s.%(ext)s'),  # Save video with its title
        'merge_output_format': 'mp4',  # Merge audio and video into mp4 format if separate
        'postprocessors': [{
            'key': 'FFmpegVideoConvertor',
            'preferedformat': 'mp4',  # Convert to mp4 if necessary
        }],
        'noplaylist': True,  # Ensure only a single video is downloaded, not playlists
    }

    try:
        with YoutubeDL(ydl_opts) as ydl:
            print(f"Downloading video from URL: {video_url}")
            ydl.download([video_url])
            print("Download complete!")
    except Exception as e:
        print(f"An error occurred: {e}")

if __name__ == "__main__":
    video_link = input("Enter the video URL: ").strip()
    output_folder = input("Enter the output directory (default: 'downloads'): ").strip() or "downloads"
    download_video(video_link, output_folder)


Enter the video URL: https://www.youtube.com/watch?v=2rtSSM-C91I
Enter the output directory (default: 'downloads'): 
[youtube] Extracting URL: https://www.youtube.com/watch?v=2rtSSM-C91I
[youtube] 2rtSSM-C91I: Downloading webpage
[youtube] 2rtSSM-C91I: Downloading ios player API JSON
[youtube] 2rtSSM-C91I: Downloading mweb player API JSON
[youtube] 2rtSSM-C91I: Downloading player 03dbdfab
[youtube] 2rtSSM-C91I: Downloading m3u8 information
[info] 2rtSSM-C91I: Downloading 1 format(s): 616+251
[hlsnative] Downloading m3u8 manifest
[hlsnative] Total fragments: 229
[download] Destination: downloads/4 Layers of Learning Every Student MUST Master.f616.mp4
[download] 100% of  357.78MiB in 00:00:40 at 8.92MiB/s                   
[download] Destination: downloads/4 Layers of Learning Every Student MUST Master.f251.webm
[download] 100% of   19.81MiB in 00:00:00 at 25.90MiB/s  
[Merger] Merging formats into "downloads/4 Layers of Learning Every Student MUST Master.mp4"
Deleting original file dow

In [3]:
# audio Clipping from video
import moviepy.editor as mp
clip = mp.VideoFileClip(r'/content/downloads/4 Layers of Learning Every Student MUST Master.mp4')
clip_audio = clip.audio.write_audiofile(r'clipped_audio.mp3')

  if event.key is 'enter':



MoviePy - Writing audio in clipped_audio.mp3


MoviePy - Done.


In [4]:
# mp3 to wav conversion
from pydub import AudioSegment
sound = AudioSegment.from_mp3("/content/clipped_audio.mp3")
sound.export("/content/audio.wav", format="wav")

<_io.BufferedRandom name='/content/audio.wav'>

In [5]:
from pydub import AudioSegment
from pydub.silence import split_on_silence
import os
import shutil
import glob

In [6]:
# chunkings
wav_file = '/content/audio.wav'
filename = wav_file.split('/')[-1].replace('.wav','')
print(filename)
audio = AudioSegment.from_wav(wav_file)
chunks = split_on_silence (
    # Use the loaded audio.
    audio,
    # Specify that a silent chunk mus t be at least 0.5 seconds or 500 ms long.
    min_silence_len = 750,
    # Consider a chunk silent if it's quieter than -16 dBFS.
    # (You may want to adjust this parameter.)
    silence_thresh = -40.0
)

audio


In [7]:
chunks

In [10]:
# saving chunks /content/chunks0.
import os
os.mkdir('/content/chunks')
for i in range(len(chunks)):
  chunks[i].export('/content/chunks/chunk'+str(i) +'.wav',format = 'wav')

In [11]:
import librosa

In [13]:
audio, rate = librosa.load("/content/chunks/chunk1.wav", sr = 16000)


In [17]:
import torch
from transformers import AutoModelForSpeechSeq2Seq, AutoProcessor, pipeline
from optimum.bettertransformer import BetterTransformer


In [18]:
device = "cuda:0" if torch.cuda.is_available() else "cpu"
torch_dtype = torch.float16 if torch.cuda.is_available() else torch.float32

In [23]:
model_id = "distil-whisper/distil-medium.en"

model = AutoModelForSpeechSeq2Seq.from_pretrained(
    model_id, torch_dtype=torch_dtype, low_cpu_mem_usage=True, use_safetensors=True#, use_flash_attention_2=True
)
model.to(device)

WhisperForConditionalGeneration(
  (model): WhisperModel(
    (encoder): WhisperEncoder(
      (conv1): Conv1d(80, 1024, kernel_size=(3,), stride=(1,), padding=(1,))
      (conv2): Conv1d(1024, 1024, kernel_size=(3,), stride=(2,), padding=(1,))
      (embed_positions): Embedding(1500, 1024)
      (layers): ModuleList(
        (0-23): 24 x WhisperEncoderLayer(
          (self_attn): WhisperSdpaAttention(
            (k_proj): Linear(in_features=1024, out_features=1024, bias=False)
            (v_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (q_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (out_proj): Linear(in_features=1024, out_features=1024, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
          (activation_fn): GELUActivation()
          (fc1): Linear(in_features=1024, out_features=4096, bias=True)
          (fc2): Linear(in_features=4096, out_features=1024, bias

In [24]:
processor = AutoProcessor.from_pretrained(model_id)

pipe = pipeline(
    "automatic-speech-recognition",
    model=model,
    tokenizer=processor.tokenizer,
    feature_extractor=processor.feature_extractor,
    max_new_tokens=128,
    chunk_length_s=15, #long form transcription
    batch_size=16,
    torch_dtype=torch_dtype,
    device=device,
)

preprocessor_config.json:   0%|          | 0.00/339 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/283k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.41M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

normalizer.json:   0%|          | 0.00/52.7k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/34.6k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.06k [00:00<?, ?B/s]

Device set to use cuda:0
  warnings.warn(



In [26]:
response = pipe("/content/chunks/chunk0.wav")

  warnings.warn(



In [27]:
import textwrap

wrapper = textwrap.TextWrapper(width=80,
    initial_indent=" " * 8,
    subsequent_indent=" " * 8,
    break_long_words=False,
    break_on_hyphens=False)

In [28]:
print(wrapper.fill(response["text"]))

         There are four layers of learning that any student or learner of any
        age should master. When we're able to master all four layers of
        learning, it is genuinely one of the most powerful learning strategies
        that you can learn. It allows us to have a better understanding and
        better retention. It makes learning faster and it even makes learning
        more enjoyable. Unfortunately, most of the students that I work with
        aren't able to follow the layers in the correct order. When this
        happens, it creates a lot of stress, it wastes a lot of time, and it can
        feel impossible to get the results you want. And with enough time, we
        can start thinking that maybe we'll just never be good at learning. But
        that probably is not true. For me, learning to learn in these four
        layers is one of the reasons I was able to get top results in both
        medical school and for my master of education. And trust me, there's
 